# Entities benchmark: data preparation for survey

Approach:
- Format data for annotation

Problems:
- Data extracted in machine format

In [5]:
# List of biographies
# 10085 - Edward Elgar
# 21511 - Niccolo Paganni
# 45181 - Clara Josephine
# 49644 - Jean Sibelious
# 50350 - Ella Jane Fitzgeral
# 57520 - Yehudi Menuhin
# 99636 - Christoph Willbald Gluck
# 180714 - Henry Wood
# 312443 - Fritz Kreisler
# 2444917 - Clara Butt

# 10085,21511, 45181,49644,50350,57520,99636,180714,312443,2444917

In [1]:
import pandas as pd
# from _datetime import date
# import nltk.tokenize as nt
# import os
# import re

In [2]:
# create the file with the required
surveyFormat_df = pd.DataFrame(columns=['IRI','sentence','file_name'])


In [81]:
files_df = pd.read_csv('../totalBiographiesEntities.csv')

# read each file in the list
for file_name in files_df.itertuples():
    print(file_name.file_name)
    # read each biography
    sentences_df = pd.read_csv('../indexedSentences/'+file_name.file_name)
    sentences_df['id'] = sentences_df["wikiId"].astype(str)+'/' + sentences_df["paragraphIndex"].astype(str) + '/' + sentences_df["sentenceIndex"].astype(str)
    sentences_df['id'] = sentences_df['id'].astype('string')
    sentences_df.drop(['sentenceIndex','paragraphIndex','section','wikiId'], inplace=True, axis=1)
    sentences_df = sentences_df[['id', 'sentences']]
    print(len(sentences_df))
    # join master biography with people
    # a. first query people
    # create query to retrive all the PEOPLE entities of the given sentence
    
    # read file with entities
    entities_df = pd.read_csv('../extractedEntitiesPersonPlaceOnly/'+file_name.file_name)
    entities_df['id'] = file_name.file_name.replace(".csv","")+'/' + entities_df['paragraphIndex'].astype(str) + '/' + entities_df["sentenceIndex"].astype(str)
    entities_df['id'] = entities_df['id'].astype('string')
    
    # people
    people_df = entities_df[entities_df['entType'] == 'person']
    people_df.drop(['support','offset','similarityScore','percentageOfSecondRank','sentence','entType',
                    'types','sentenceIndex','paragraphIndex','section','wikiPageID'], inplace=True, axis=1)
    people_df=people_df.rename(columns = {'entity':'people_surface_form','URI':'people_dbpedia'})
    people_df = people_df[['people_surface_form', 'people_dbpedia','id']]
    print(len(people_df))
    
    new_df_peple = sentences_df.merge(people_df,on=['id'],how="inner")#,lsuffix="_m",rsuffix="_p")
    print(len(new_df_peple))
    new_df_peple.to_csv('sample/people_'+file_name.file_name,index=False)    
    
    
    # place
    place_df = entities_df[entities_df['entType'] == 'place']
    place_df.drop(['support','offset','similarityScore','percentageOfSecondRank','sentence','entType',
                    'types','sentenceIndex','paragraphIndex','section','wikiPageID'], inplace=True, axis=1)
    place_df=place_df.rename(columns = {'entity':'place_surface_form','URI':'place_dbpedia'})
    place_df = place_df[['place_surface_form', 'place_dbpedia','id']]
    # place_df.to_csv('sample/place_'+file_name.file_name,index=False)    
    
    print(len(place_df))
    new_df_place = sentences_df.merge(place_df,on=['id'],how="inner")#,lsuffix="_m",rsuffix="_p")
    print(len(new_df_place))
    new_df_place.to_csv('sample/place_'+file_name.file_name,index=False)    
    
    time_file = False
    try:
            time_df = pd.read_csv('../extractedTimeExpressions/'+file_name.file_name)
            time_df['id'] = time_df["wikiId"].astype(str)+'/' + time_df['paragraphIndex'].astype(str) + '/' + time_df["sentenceIndex"].astype(str)
            time_df['id'] = time_df['id'].astype('string')
            time_df.drop(['POST_tag','time_token','type_token','sentences',
                          'sentenceIndex','paragraphIndex','section','wikiId'], inplace=True, axis=1)    
            new_df_time = sentences_df.merge(time_df,on=['id'],how="inner")#,lsuffix="_m",rsuffix="_p")
            print(len(new_df_time))
            new_df_time.to_csv('sample/time_'+file_name.file_name,index=False)
            time_file = True
    except:
        time_file = False
    
    new_df = pd.DataFrame()
    for sentence in sentences_df.itertuples():
        query_place = "id=='"+ str(sentence.id)+"'"
        # print(query_place)
        new_ppl = new_df_peple.query(query_place)
        new_df = new_df.append(new_ppl)
        new_plc = new_df_place.query(query_place)
        new_df = new_df.append(new_plc, ignore_index = True)
        if time_file:
            new_time = new_df_time.query(query_place)
            new_df = new_df.append(new_time, ignore_index = True)
            
    sentences_df['annotation'] = 'N'
    new_df = sentences_df.merge(new_df,on=['id'],how="left")#,lsuffix="_m",rsuffix="_p")
    new_df.drop(['sentences_y'], inplace=True, axis=1)
    new_df=new_df.rename(columns = {'sentences_x':'sentence'})
    
    new_df.to_csv('sample/'+file_name.file_name,index=False)    
    print("done")

10085.csv
460
473
473
153
153


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


255
done
21511.csv
263
178
178
108
108
done
45181.csv
392
636
636
192
192
done
49644.csv
563
900
900
534
534
done
50350.csv
290
678
678
158
158
done
57520.csv
182
298
298
166
166
done
99636.csv
365
385
385
155
155
done
180714.csv
364
554
554
190
190
done
312443.csv
142
218
218
60
60
done
2444917.csv
47
100
100
46
46
done


In [80]:
time_df.info()
time_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   time_expression  255 non-null    object
 1   id               255 non-null    string
dtypes: object(1), string(1)
memory usage: 4.1+ KB


,time_expression,id
0,2 June 1857 – 23 February 1934,10085/0/0
1,1924,10085/0/3
2,some quarters,10085/1/2
3,forties,10085/1/4
4,1899,10085/1/4


In [44]:
sentences_df.info()
sentences_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentences  460 non-null    object
 1   id         460 non-null    string
dtypes: object(1), string(1)
memory usage: 7.3+ KB


,sentences,id
0,"Sir Edward William Elgar, 1st Baronet, ( (lis...",10085/0/0
1,Among his best-known compositions are orchestr...,10085/0/1
2,"He also composed choral works, including The D...",10085/0/2
3,He was appointed Master of the King's Musick i...,10085/0/3
4,Although Elgar is often regarded as a typicall...,10085/1/0


In [48]:
people_df.info()
people_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 630
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   people_dbpedia  473 non-null    object
 1   surface_form    473 non-null    object
 2   id              473 non-null    string
dtypes: object(2), string(1)
memory usage: 14.8+ KB


,people_dbpedia,surface_form,id
0,http://dbpedia.org/resource/Saint_George,St George,10085/6/6
1,http://dbpedia.org/resource/Johann_Nepomuk_Hummel,Hummel,10085/6/7
2,http://dbpedia.org/resource/Ludwig_van_Beethoven,Beethoven,10085/7/29
3,http://dbpedia.org/resource/Madeleine_of_Valois,Madeleine,10085/8/1
4,http://dbpedia.org/resource/Robert_Schumann,Schumann,10085/8/2


In [6]:
# df_files = pd.DataFrame(files_list, columns=['file_name'])
# # df_files = df_files.query("file_name=='10085.txt'")
# df_files.to_csv('totalBiographies.csv',index=False)
files_df = pd.read_csv('../totalBiographiesEntities.csv')

# read each file in the list
for file_name in files_df.itertuples():
    print(file_name.file_name)
    # read each biography
    sentences_df = pd.read_csv('../indexedSentences/'+file_name.file_name)
    
    # create survey dataframe
    df_master = pd.DataFrame(columns=['IRI','sentence','people_ent','people_le','subject','place_ent',
                                  'place_le'])
    # for each sentence in the biography
    for sentence_line in sentences_df.itertuples():
        df_new_rows = pd.DataFrame()
        # create query to retrive all the PEOPLE entities of the given sentence
        query = "paragraphIndex=="+ str(sentence_line.paragraphIndex) +" & sentenceIndex=="+str(sentence_line.sentenceIndex)+" & entType=='person'"
        print(query) 
        # read file with entities
        entities_df = pd.read_csv('../extractedEntitiesPersonPlaceOnly/'+file_name.file_name)
        entities_df = entities_df.query(query)
        # print(len(entities_df))
        # if the given sentence has one or more entities
        if len(entities_df)>0:
            # print(len(entities_df))
            # print("person found sentence")
            df_new_rows['sentence'] = entities_df['sentence']
            df_new_rows['IRI'] = str(sentence_line.wikiId)+'/' + str(sentence_line.paragraphIndex) + '/' + str(sentence_line.sentenceIndex)
            df_new_rows['people_ent'] = entities_df['entity']
            df_new_rows['people_le'] = entities_df['URI']
            # print(df_new_rows)
        else: 
            # create query to retrive all the PEOPLE entities of the given sentence
            query = "paragraphIndex=="+ str(sentence_line.paragraphIndex) + " & sentenceIndex=="+str(sentence_line.sentenceIndex) + " & entType=='place'"
            # print(query) 
            # read file with entities
            entities_df = pd.read_csv('../extractedEntitiesPersonPlaceOnly/'+file_name.file_name)
            entities_df = entities_df.query(query)
            if len(entities_df)>0:
                # print(len(entities_df))
                # print("person found sentence")
                df_new_rows['sentence'] = entities_df['sentence']
                df_new_rows['IRI'] = str(sentence_line.wikiId)+'/' + str(sentence_line.paragraphIndex) + '/' + str(sentence_line.sentenceIndex)
                df_new_rows['place_ent'] = entities_df['entity']
                df_new_rows['place_le'] = entities_df['URI']
                # print(df_new_rows)
            else:
                # print("NO person found sentence")
                df_master = df_master.append({'IRI':str(sentence_line.wikiId)+'/' + str(sentence_line.paragraphIndex) + '/' + str(sentence_line.sentenceIndex),
                                             'sentence':sentence_line.sentences}, ignore_index = True)
                # df_new_rows['IRI'] = str(sentence_line.wikiId)+'/' + str(sentence_line.paragraphIndex) + '/' + str(sentence_line.sentenceIndex)
                # df_new_rows['sentence'] = sentence_line.sentences
                # print(df_new_rows)
        df_master = df_master.append(df_new_rows, ignore_index = True)
        # print(len(df_master))
        
        # print((df_master))
            
    # print(len(df_master))
    df_master.info()
        # df_new_rows['subject'] = 
    # # biography_df.sort_values(by=['paragraphIndex','sentenceIndex'], ascending=False, inplace=True)
    # biography_df.rename(columns={'URI':'IRI'}, inplace=True)
    # biography_df=biography_df.reindex(columns= ['IRI', 'sentence', 'Score','Name'])    

10085.csv
paragraphIndex==0 & sentenceIndex==0 & entType=='person'
paragraphIndex==0 & sentenceIndex==1 & entType=='person'
paragraphIndex==0 & sentenceIndex==2 & entType=='person'
paragraphIndex==0 & sentenceIndex==3 & entType=='person'
paragraphIndex==1 & sentenceIndex==0 & entType=='person'
paragraphIndex==1 & sentenceIndex==1 & entType=='person'
paragraphIndex==1 & sentenceIndex==2 & entType=='person'
paragraphIndex==1 & sentenceIndex==3 & entType=='person'
paragraphIndex==1 & sentenceIndex==4 & entType=='person'
paragraphIndex==1 & sentenceIndex==5 & entType=='person'
paragraphIndex==1 & sentenceIndex==6 & entType=='person'
paragraphIndex==2 & sentenceIndex==0 & entType=='person'
paragraphIndex==2 & sentenceIndex==1 & entType=='person'
paragraphIndex==2 & sentenceIndex==2 & entType=='person'
paragraphIndex==2 & sentenceIndex==3 & entType=='person'
paragraphIndex==2 & sentenceIndex==4 & entType=='person'
paragraphIndex==2 & sentenceIndex==5 & entType=='person'
paragraphIndex==3 & s

In [8]:
df_master.to_csv('sampleEntities/10085.csv',index=False)


In [7]:
biography_df.info()
biography_df.head()

NameError: name 'biography_df' is not defined